<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jjc/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

In [ ]:
doc_ls = []
sent = '바나나 사과 포도 포도 짜장면'
doc_ls.append(sent)
sent = '사과 포도'
doc_ls.append(sent)
sent = '포도 바나나'
doc_ls.append(sent)
sent = '짜장면 짬뽕 탕수육'
doc_ls.append(sent)
sent = '볶음밥 탕수육'
doc_ls.append(sent)
sent = '짜장면 짬뽕'
doc_ls.append(sent)
sent = '라면 스시'
doc_ls.append(sent)
sent = '스시 짜장면'
doc_ls.append(sent)
sent = '가츠동 스시 소바'
doc_ls.append(sent)
sent = '된장찌개 김치찌개 김치'
doc_ls.append(sent)
sent = '김치 된장 짜장면'
doc_ls.append(sent)
sent = '비빔밥 김치'
doc_ls.append(sent)


In [ ]:
doc_ls

['바나나 사과 포도 포도 짜장면',
 '사과 포도',
 '포도 바나나',
 '짜장면 짬뽕 탕수육',
 '볶음밥 탕수육',
 '짜장면 짬뽕',
 '라면 스시',
 '스시 짜장면',
 '가츠동 스시 소바',
 '된장찌개 김치찌개 김치',
 '김치 된장 짜장면',
 '비빔밥 김치']

In [505]:
import numpy as np
import itertools
from sklearn.decomposition import randomized_svd

class LSA():
    a = 0

def tdm(doc_list):
    # TDM 생성
    tokens = []
    for doc in doc_list:
        tokens.append(doc.split())
    total_token = list(itertools.chain(*tokens))
    unique_token = set(total_token)
    tdm_df = pd.DataFrame(unique_token, columns=['word'])
    
    for i, dt in enumerate(tokens):
        doc_count = []
        for ut in unique_token:
            doc_count.append(dt.count(ut))
        tdm_df[i] = doc_count

    tdm_df.set_index('word',inplace=True)
    return tdm_df

def lsa(tdm, dim):
    # LSA 생성
    tdm_matrix = np.array(tdm.T) # 문서-단어 행렬
    U, s, VT = randomized_svd(tdm_matrix,
                              n_components=dim,
                              n_iter=10,
                              random_state=None)
    # U, s, VT = np.linalg.svd(tdm_matrix) # 문서-주제 행렬, 주제 정보량(특이값), 주제-단어 행렬
    return U, s, VT
    # return U[:, :dim], s[:dim], np.matrix(VT[:dim, :])

def topic_modeling(tdm, VT, word_count):
    # Topic Modeling
    tm_df = pd.DataFrame(VT).T.set_index(tdm.index)
    print('--- Topic Modeling ---')
    for i in tm_df:
        print('Topic {} - '.format(i+1), end='')
        indexs = list(tm_df.sort_values(by=i, ascending=False)[i][:word_count].index)
        values = list(map(lambda i : round(i, 3), tm_df.sort_values(by=i, ascending=False)[i][:word_count].values))
        print(dict(zip(indexs,values)))

def word_sim(tdm, VT, *words):
    ws_df = pd.DataFrame(VT).T.set_index(tdm.index).T
    print('---word similarity---')

    final_result = []
    for word1 in words:
        a = np.sqrt((np.array(ws_df[word1])**2).sum())
        temp = []
        for word2 in words:
            b = np.sqrt((np.array(ws_df[word2])**2).sum())
            temp.append(float(np.matrix(ws_df[word1]).dot(np.matrix(ws_df[word2]).T)/(a * b)))
        final_result.append(temp)
    return pd.DataFrame(final_result, index = words, columns = words)

def doc_sim(tdm, U, *docs):
    ds_df = pd.DataFrame(U, columns=['Topic_{}'.format(i) for i in range(1,len(U.T)+1)])
    print('---docs similarity---')

    final_result = []
    for doc1 in docs:
        a = np.sqrt((np.array(ds_df.T[doc1])**2).sum())
        temp = []
        for doc2 in docs:
            b = np.sqrt((np.array(ds_df.T[doc2])**2).sum())
            temp.append(float(np.matrix(ds_df.T[doc1]).dot(np.matrix(ds_df.T[doc2]).T)/(a * b)))
        final_result.append(temp)
    return pd.DataFrame(final_result, index = docs, columns = docs)

def doc_word_sim():
    return

In [506]:
tdm_df = tdm(doc_ls)

In [507]:
tdm_df

,0,1,2,3,4,5,6,7,8,9,10,11
word,,,,,,,,,,,,
라면,0,0,0,0,0,0,1,0,0,0,0,0
김치찌개,0,0,0,0,0,0,0,0,0,1,0,0
탕수육,0,0,0,1,1,0,0,0,0,0,0,0
된장찌개,0,0,0,0,0,0,0,0,0,1,0,0
사과,1,1,0,0,0,0,0,0,0,0,0,0
볶음밥,0,0,0,0,1,0,0,0,0,0,0,0
김치,0,0,0,0,0,0,0,0,0,1,1,1
소바,0,0,0,0,0,0,0,0,1,0,0,0
스시,0,0,0,0,0,0,1,1,1,0,0,0


In [508]:
u, s, vt = lsa(tdm_df, 4)

In [509]:
topic_modeling(tdm_df, vt, 3)

--- Topic Modeling ---
Topic 1 - {'포도': 0.697, '짜장면': 0.486, '사과': 0.348}
Topic 2 - {'짜장면': 0.584, '짬뽕': 0.356, '김치': 0.337}
Topic 3 - {'김치': 0.611, '김치찌개': 0.264, '된장찌개': 0.264}
Topic 4 - {'스시': 0.552, '김치': 0.371, '소바': 0.277}


In [510]:
word_sim(tdm_df, vt, '김치', '김치찌개', '탕수육')

---word similarity---


,김치,김치찌개,탕수육
김치,1.000000,0.978699,-0.251469
김치찌개,0.978699,1.000000,-0.436448
탕수육,-0.251469,-0.436448,1.000000


In [511]:
doc_sim(tdm_df, u, 0,1,3)

---docs similarity---


,0,1,3
0,1.000000,0.934065,0.078413
1,0.934065,1.000000,-0.261173
3,0.078413,-0.261173,1.000000
